In [6]:
import re

class Morph():
    def __init__(self, doc_list):
        self.surface = doc_list[0]
        self.base = doc_list[7]
        self.pos = doc_list[1]
        self.pos1= doc_list[2]

    def __str__(self):
        return "{" + f"surface: {self.surface}, base: {self.base}, pos: {self.pos}, pos1: {self.pos1}" + "}"

class Chunk():
    def __init__(self,morph_list,dst,srcs):
        self.morph_list = morph_list
        self.dst = int(dst)
        self.srcs = srcs

    def __str__(self):
        morphs_str = ', '.join(str(morph) for morph in self.morph_list) # Morphオブジェクトのリストを文字列に変換
        return f"Morphs: [{morphs_str}], Dst: {self.dst}, Srcs: {self.srcs}"

# 文節番号と係り先番号の取得
def find_feature_number(text):
    pattern = r"^\* \d+ ([^\d]*\d+)D"
    match = re.search(pattern, text)
    if match:
        dst_number = match.group(1)  # 文節番号  # 係先番号
        return dst_number
    else:
        return None

In [7]:
with open(file="ai.ja.txt.parsed", mode="r", encoding="utf-8")as f:
    docs = f.readlines()

Chunk_list = []
chunk_flag = False
# srcの追加処理は後で行う
for doc in docs[5:]:
    if doc == "EOS\n":
        chunk = Chunk(morph_list,dst_num,[])
        Chunk_list.append(chunk)
        break
    # *から始まる行とEOSの行は飛ばす
    if doc[0]=="*":
        if chunk_flag:
            chunk = Chunk(morph_list,dst_num,[])
            Chunk_list.append(chunk)
        dst_num = find_feature_number(doc)
        morph_list = []
        chunk_flag = True
    else:
        doc = doc.replace("\t",",").replace("\n", "")
        doc_list = doc.split(",")
        doc_morph = Morph(doc_list)
        morph_list.append(doc_morph)


# Srcsを埋めていく
for i in range(len(Chunk_list)):
    Chunk_list[Chunk_list[i].dst].srcs.append(i)

In [25]:
# 実装開始
# 文節に名詞を含んでいるかの判定
def has_noun_in_sentence(chunk: Chunk) -> bool:
    for morph in chunk.morph_list:
        if morph.pos == "名詞":
            return True
    return False

#名詞をXとYのどちらかに変更する
def change_X_or_Y(chunk,moji):
    sentence = ""
    remove_list = ["（", "）", "、", "。", "「", "」", "『", "』", "〈", "〉"]
    change_flag = True
    for morph in chunk.morph_list:
        if morph.pos == "名詞":
            if change_flag:
                sentence += moji
                change_flag = False
            else:
                continue
        elif morph.surface not in remove_list:
            sentence += morph.surface
    return sentence

def get_sentence_from_chunk(chunk):
    remove_list = ["（", "）", "、", "。", "「", "」", "『", "』", "〈", "〉"]
    sentence = ""
    for morph in chunk.morph_list:
        if morph.surface not in remove_list:
            sentence += morph.surface
    return sentence


#YがXから根までのます上にあればX -> Yという形で出力する，なければNone
def Y_on_path_of_X(i,j,chunk_list):
    path_text = change_X_or_Y(chunk_list[i], "X")
    dst = chunk_list[i].dst
    while dst != -1:
        if dst == j:
            path_text += "->"
            path_text += change_X_or_Y(chunk_list[j], "Y")
            return path_text
        else:
            path_text += "->"
            path_text += get_sentence_from_chunk(chunk_list[dst])
            dst = chunk_list[dst].dst
    return None

# chunkから根までの文節番号のリストを返す
# 例えば文節番号が1 -> 3 -> 5 -> -1だったら[1, 3, 5, -1]を返す
# 引数は文節番号
def get_path_chunk_list(i, chunnk_list):
    dst = chunnk_list[i].dst
    dst_list = [i]
    while dst != -1:
        dst_list.append(dst)
        dst = chunnk_list[dst].dst
    dst_list.append(-1)
    return dst_list

#一致する文節番号があればその文節番号(最初に一致した合流点)を，なければNoneを返す」
def has_same_chunk(i_path_list, j_path_list):
    for i_path in i_path_list:
        for j_path in j_path_list:
            if i_path == j_path:
                return i_path
    return None

#途中でパスがぶつかっている場合の処理
# chunk_numの直前までを出力する
def print_path_X_and_Y(i_path_list, j_path_list, same_chunk_num, chunk_list):
    first_flag = True
    for i_path in i_path_list:
        if i_path == same_chunk_num:
            break
        if first_flag:
            i_chunk = chunk_list[i_path]
            print(change_X_or_Y(i_chunk, "X"), end = "")
            first_flag = False
        else:
            sentence = get_sentence_from_chunk(chunk_list[i_path])
            print(f" -> {sentence}", end = "")
    print(" | ", end = "")
    first_flag = True
    for j_path in j_path_list:
        if j_path == same_chunk_num:
            break
        if first_flag:
            j_chunk = chunk_list[j_path]
            print(change_X_or_Y(j_chunk, "Y"), end = "")
            first_flag = False
        else:
            sentence = get_sentence_from_chunk(chunk_list[j_path])
            print(f" -> {sentence}", end="")
    print(" | ", end = "")
    print(get_sentence_from_chunk(chunk_list[same_chunk_num]))

for i in range(len(Chunk_list)):
    if has_noun_in_sentence(Chunk_list[i]):
        for j in range(i+1,len(Chunk_list)):
            if has_noun_in_sentence(Chunk_list[j]):
                path_text = Y_on_path_of_X(i, j, Chunk_list)
                if path_text != None:
                    print(path_text)
                    continue
                i_path_list = get_path_chunk_list(i, Chunk_list)
                j_path_list = get_path_chunk_list(j, Chunk_list)
                same_chunk_num = has_same_chunk(i_path_list, j_path_list)
                if same_chunk_num != None:
                    print_path_X_and_Y(i_path_list, j_path_list, same_chunk_num, Chunk_list)




X | Yのう | 語
X | Y -> エーアイとは | 語
X | Yとは | 語
X | Y -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す | 語
X | Yと -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す | 語
X | Y -> という -> 道具を -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す | 語
X | Yを -> 用いて -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す | 語
X | Yを -> 研究する -> 計算機科学 -> の -> 一分野を -> 指す | 語
X | Yする -> 計算機科学 -> の -> 一分野を -> 指す | 語
X | Y -> の -> 一分野を -> 指す | 語
X | Yを -> 指す | 語
X->Y
X -> 語 | Yの -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または | 研究分野とも
X -> 語 | Yや -> 推論 -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または | 研究分野とも
X -> 語 | Y -> 問題解決などの -> 知的行動を -> 代わって -> 行わせる -> 技術または | 研究分野とも
X -> 語 | Yなどの -> 知的行動を -> 代わって -> 行わせる -> 技術または | 研究分野とも
X -> 語 | Yを -> 代わって -> 行わせる -> 技術または | 研究分野とも
X -> 語 | Yに -> 代わって -> 行わせる -> 技術または | 研究分野とも
X -> 語 | Yに -> 行わせる -> 技術または | 研究分野とも
X -> 語 | Yまたは | 研究分野とも
X -> 語 | Y -> コンピュータによる -> 情報処理システムの -> 実現に関する | 研究分野とも
X -> 語 | Yによる -> 情報処理システムの -> 実現に関する | 研究分野とも
X -> 語 | Yな -> 情報処理システムの -> 実現に関する | 研究分野とも
X ->